In [99]:
from xbbg import blp
import numpy as np
import pandas as pd
from datetime import datetime, date
from scipy.stats import qmc, norm
import os

In [19]:
option_ticker = 'MSFT US 08/18/23 C320 Equity'
ticker = 'MSFT US Equity'
df = blp.bdh(tickers=[option_ticker, ticker], flds=['last_price', 'IVOL_MID'], start_date='2023-05-01', end_date='2023-08-15', adjust='all')

In [96]:
df_final = df[option_ticker].rename(columns = {'last_price': 'option_price', 'IVOL_MID':'sigma_impl'}).copy()
df_final['asset_price'] = df[ticker].rename(columns = {'last_price': 'asset_price'})['asset_price'].copy()
df_final['option_ticker'] = option_ticker
df_final['ticker'] = ticker
df_final['K'] = 320.0
df_final['expiry'] = date(2023,8,18)
df_final['r'] = 0.02
df_final['t'] = df_final.apply(lambda row: (row.expiry - row.name).days / 360.0, axis=1)
df_final

,option_price,sigma_impl,asset_price,option_ticker,ticker,K,expiry,r,t
2023-05-01,11.62,23.619,304.8936,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.302778
2023-05-02,11.87,24.451,304.7439,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.300000
2023-05-03,11.37,24.593,303.7361,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.297222
2023-05-04,12.74,25.510,304.7439,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.294444
2023-05-05,14.10,24.573,309.9725,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.291667
...,...,...,...,...,...,...,...,...,...
2023-08-09,6.54,25.227,322.2300,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.025000
2023-08-10,6.35,24.182,322.9300,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.022222
2023-08-11,4.28,22.325,321.0100,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.019444
2023-08-14,5.50,20.097,324.0400,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.011111


In [113]:
def get_d(S, sigma, div, r, K, dt):
    d1 = (np.log(S / K) + (r - div + 0.5 * sigma**2) * dt) / sigma / np.sqrt(dt)
    d2 = d1 - sigma * np.sqrt(dt)
    return d1, d2

def get_N(S, sigma, div, r, K, dt):
    d1, d2 = get_d(S, sigma, div, r, K, dt)
    return norm.cdf(d1), norm.cdf(d2)

def get_deltaN(d):
    return (1.0 / np.sqrt(2*np.pi))*np.exp(-d**2/2)

def get_call(S, sigma, div, r, K, dt):
    N1, N2 = get_N(S, sigma, div, r, K, dt)
    return S * np.exp(-div * dt) * N1 - K * np.exp(-r * dt) * N2

def get_delta(S, sigma, div, r, K, dt):
    N1, N2 = get_N(S, sigma, div, r, K, dt)
    return np.exp(-div * dt) * N1

def get_gamma(S, sigma, div, r, K, dt):
    if dt == 0:
        return 0
    d1, d2 = get_d(S, sigma, div, r, K, dt)
    return np.exp(-div * dt) * get_deltaN(d1) / sigma / S / np.sqrt(dt)

def get_vega(S, sigma, div, r, K, dt):
    if dt == 0:
        return 0
    d1, d2 = get_d(S, sigma, div, r, K, dt)
    return S * np.sqrt(dt) * np.exp(-div * dt) * get_deltaN(d1)

In [119]:
df_final['call_option'] = df_final.apply(lambda row: get_call(row.asset_price, row.sigma_impl/100.0, 0, row.r, row.K, row.t), axis=1)
df_final['D_call_option'] = df_final.call_option - df_final.call_option.shift(1)
df_final['D_sigma_impl'] = df_final.sigma_impl - df_final.sigma_impl.shift(1)
df_final['sqrt_t'] = np.sqrt(df_final.t)
df_final['D_S'] = df_final.asset_price - df_final.asset_price.shift(1)
df_final['delta_bs'] = df_final.apply(lambda row: get_delta(row.asset_price, row.sigma_impl/100.0, 0, row.r, row.K, row.t), axis=1)
df_final['vega_bs'] = df_final.apply(lambda row: get_vega(row.asset_price, row.sigma_impl/100.0, 0, row.r, row.K, row.t), axis=1)
df_final['y_serie'] = df_final.apply(lambda row: (row.sqrt_t * row.asset_price*(row.D_call_option - row.delta_bs*row.D_S))/(row.vega_bs*row.D_S), axis=1)
df_final

,option_price,sigma_impl,asset_price,option_ticker,ticker,K,expiry,r,t,call_option,D_sigma_impl,sqrt_t,D_S,delta_bs,D_call_option,vega_bs,y_serie
2023-05-01,11.62,23.619,304.8936,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.302778,10.397202,NaN,0.550252,NaN,0.397234,NaN,64.696879,NaN
2023-05-02,11.87,24.451,304.7439,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.300000,10.797242,0.832,0.547723,-0.1497,0.400138,0.400040,64.492220,-7.951844
2023-05-03,11.37,24.593,303.7361,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.297222,10.410576,0.142,0.545181,-1.0078,0.390633,-0.386666,63.563065,-0.018129
2023-05-04,12.74,25.510,304.7439,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.294444,11.315561,0.917,0.542627,1.0078,0.404723,0.904985,64.079617,1.272890
2023-05-05,14.10,24.573,309.9725,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.291667,12.849303,-0.937,0.540062,5.2286,0.448444,1.533743,66.226193,-0.392074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-09,6.54,25.227,322.2300,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.025000,6.391898,1.554,0.158114,-3.8200,0.581832,-2.400014,19.896601,0.118927
2023-08-10,6.35,24.182,322.9300,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.022222,6.319414,-1.045,0.149071,0.7000,0.611487,-0.072484,18.450025,-1.865662
2023-08-11,4.28,22.325,321.0100,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.019444,4.572447,-1.857,0.139443,-1.9200,0.551434,-1.746967,17.709116,0.906028
2023-08-14,5.50,20.097,324.0400,MSFT US 08/18/23 C320 Equity,MSFT US Equity,320.0,2023-08-18,0.02,0.011111,5.256464,-2.228,0.105409,3.0300,0.730167,0.684017,11.290306,-1.526028


In [104]:
df_final.sigma_impl.shift(1) - df_final.sigma_impl

2023-05-01      NaN
2023-05-02   -0.832
2023-05-03   -0.142
2023-05-04   -0.917
2023-05-05    0.937
              ...  
2023-08-09   -1.554
2023-08-10    1.045
2023-08-11    1.857
2023-08-14    2.228
2023-08-15      NaN
Name: sigma_impl, Length: 74, dtype: float64

In [60]:
df_final.to_csv('data.csv', index=False)